<h1 style="color:rgb(0,120,170)">Neural Networks and Deep Learning</h1>
<h2 style="color:rgb(0,120,170)">Transfer Learning</h2>

In [7]:
#!pip install -q -U tensorflow
#!pip install tqdm

### Downloading the Dogs vs Cats dataset 

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

In [6]:
import requests
url = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
r = requests.get(url, allow_redirects=True)
open('cats_and_dogs_filtered.zip', 'wb').write(r.content)

68606236

## Dataset preprocessing

### Import project dependencies

In [8]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.3.1'

### Unzipping the Dogs vs Cats dataset

In [9]:
dataset_path = "./cats_and_dogs_filtered.zip"
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")
zip_object.extractall("./")
zip_object.close()

### Seting up dataset paths

In [13]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [14]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model [MobileNetV2](https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2) from [Keras Applications](https://www.tensorflow.org/api_docs/python/tf/keras/applications)  

In [15]:
IMG_SHAPE = (128, 128, 3)

In [16]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [17]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Freezing the base model

In [18]:
base_model.trainable = False

### Defining the custom head for our network

In [19]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [20]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [21]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [22]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [23]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [24]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________________

### Compiling the model

In [25]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [26]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [27]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [28]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [29]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 22s 1s/step - loss: 0.8190 - accuracy: 0.4755 - val_loss: 0.7437 - val_accuracy: 0.5290
Epoch 2/5
16/16 [==============================] - 23s 1s/step - loss: 0.6921 - accuracy: 0.5955 - val_loss: 0.6461 - val_accuracy: 0.6410
Epoch 3/5
16/16 [==============================] - 22s 1s/step - loss: 0.6026 - accuracy: 0.6815 - val_loss: 0.5651 - val_accuracy: 0.7340
Epoch 4/5
16/16 [==============================] - 23s 1s/step - loss: 0.5262 - accuracy: 0.7500 - val_loss: 0.4961 - val_accuracy: 0.7980
Epoch 5/5
16/16 [==============================] - 23s 1s/step - loss: 0.4616 - accuracy: 0.8090 - val_loss: 0.4378 - val_accuracy: 0.8320


### Transfer learning model evaluation

In [30]:
valid_loss, valid_accuracy = model.evaluate(valid_generator)

8/8 [==============================] - 6s 760ms/step - loss: 0.4378 - accuracy: 0.8320


In [31]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8320000171661377


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [32]:
base_model.trainable = True

In [33]:
print("Number of layers in the base model: {}".format(len(base_model.layers)))

Number of layers in the base model: 155


In [34]:
fine_tune_at = 100

In [35]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [36]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [37]:
model.fit(train_generator, epochs=5, validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 40s 3s/step - loss: 0.2081 - accuracy: 0.9170 - val_loss: 0.0968 - val_accuracy: 0.9660
Epoch 2/5
16/16 [==============================] - 39s 2s/step - loss: 0.0289 - accuracy: 0.9980 - val_loss: 0.1079 - val_accuracy: 0.9660
Epoch 3/5
16/16 [==============================] - 38s 2s/step - loss: 0.0125 - accuracy: 0.9995 - val_loss: 0.1182 - val_accuracy: 0.9680
Epoch 4/5
16/16 [==============================] - 39s 2s/step - loss: 0.0036 - accuracy: 1.0000 - val_loss: 0.1471 - val_accuracy: 0.9650
Epoch 5/5
16/16 [==============================] - 39s 2s/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.1395 - val_accuracy: 0.9690


### Evaluating the fine tuned model

In [38]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [39]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.968999981880188
